# Named Entity Recognition
- using ML base model

# 1)- Importing key modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [0]:
# For data processing and maths

import pandas as pd
import numpy as np
#For Visuals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [3]:
! pip install version_information

In [4]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,numpy,seaborn, matplotlib

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.25.3
numpy 1.17.4
seaborn 0.9.0
matplotlib 3.1.1
Tue Nov 26 19:33:42 2019 UTC

# 2)- Getting Data

https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus

In [0]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [6]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


There are missing values. We shall fill wit ffill. Also it makes sense as sentence 1 contains all words and then comes sentence2.

In [0]:
data = data.fillna(method="ffill")

In [8]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [9]:
data.tail()

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


So, in total we have 47959 sentences in our data. Let's see how many words we have

In [10]:
len(data["Word"])

1048575

Total words are 1048575. But , there might be many repeating words. Let's find unique words or vocab.

In [11]:
len(data["Word"].unique())

35178

In [0]:
# making list of unique "word" feature
words = list(set(data["Word"].values))

In [13]:
words[:5]

['15-second', 'fortune', 'diesel', 'lesions', '5']

In [14]:
n_words = len(words)
n_words

35178

We have 47959 sentences containing 35178 unique words

In [15]:
# let's see how many tag we have in our data or what kind of tags we have

data.Tag.value_counts()

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

In [16]:
len(data.Tag.value_counts())

17

In [17]:
# how about POS

len(data.POS.value_counts())

42

#3)- Retrieving sentence

In [0]:
d=data

In [0]:
d=d.groupby(["Sentence #","Tag"]).size()

In [20]:
d.head()

Sentence #    Tag  
Sentence: 1   B-geo     2
              B-gpe     1
              O        21
Sentence: 10  B-gpe     1
              B-org     1
dtype: int64

In [0]:
d_tag=d.unstack('Tag')

In [22]:
d_tag

Tag,B-art,B-eve,B-geo,B-gpe,B-nat,B-org,B-per,B-tim,I-art,I-eve,I-geo,I-gpe,I-nat,I-org,I-per,I-tim,O
Sentence #,,,,,,,,,,,,,,,,,
Sentence: 1,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
Sentence: 10,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
Sentence: 100,NaN,NaN,2.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,27.0
Sentence: 1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0
Sentence: 10000,NaN,NaN,3.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sentence: 9995,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,27.0
Sentence: 9996,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,21.0
Sentence: 9997,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,24.0


Thanks to blog by http://www.depends-on-the-definition.com/introduction-named-entity-recognition-python/

We can use a simpler solution for this 

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
    
    def get_next(self):
        try:
            s = self.data[self.data["Sentence #"] == "Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s["Word"].values.tolist(), s["POS"].values.tolist(), s["Tag"].values.tolist()    
        except:
            self.empty = True
            return None, None, None

In [0]:
getter = SentenceGetter(data)

In [0]:
sent, pos, tag = getter.get_next()

In [26]:
print(sent)

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']


In [27]:
print(pos)

['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'CC', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.']


In [28]:
 print(tag)

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


O is for the word that we don't know so, we predict it as "O"

# 4)- Base model

In [29]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [30]:
words_counts = {}

from collections import Counter
words_counts = Counter([word for line in data['Word'] for word in line.split(' ')])

# Sorting 
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:10]

# Top 10
most_common_words[:10]

[('the', 52573),
 ('.', 47761),
 (',', 32754),
 ('of', 26354),
 ('in', 26323),
 ('to', 23213),
 ('a', 20481),
 ('and', 19936),
 ('The', 11313),
 ("'s", 10923)]

In [0]:
#Splitting data


X=data['Word']
y=data['Tag']

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(786431,)
(262144,)
(786431,)
(262144,)


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_features(X_train, X_test):
    """
        X_train, X_val, X_test - input text       
        return TF-IDF vectorizer for each dataset
    """
    
    # filter out too rare words (occur less than in 5 titles) and too frequent words (occur more than in 90% of the tweets)
    # ngram!!! -->  ngram_range=(1,2)
    tfidf_vectorizer = TfidfVectorizer(max_features= 500, max_df=0.9, min_df=5, token_pattern='(\S+)')
    
    # Fit and transform the vectorizer on the train set
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    
    # Only Transform the test sets 
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    return X_train_tfidf, X_test_tfidf, tfidf_vectorizer.vocabulary_
    
    
X_train_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_test)

In [0]:

# convert to dense array
X_train_tfidf=X_train_tfidf.toarray()

In [35]:

type(X_train_tfidf)

numpy.ndarray

In [0]:
X_test_tfidf=X_test_tfidf.toarray()

In [0]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train_tfidf,y_train)
predmnb = mnb.predict(X_test_tfidf)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predmnb)

0.8945159912109375

In [39]:
from sklearn.metrics import classification_report
print (classification_report(y_test, predmnb))

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        97
       B-eve       0.00      0.00      0.00        70
       B-geo       0.80      0.49      0.61      9436
       B-gpe       0.95      0.49      0.65      4000
       B-nat       0.00      0.00      0.00        55
       B-org       0.73      0.21      0.32      5056
       B-per       0.85      0.38      0.52      4194
       B-tim       0.93      0.56      0.70      5188
       I-art       0.00      0.00      0.00        72
       I-eve       0.00      0.00      0.00        70
       I-geo       0.73      0.35      0.47      1783
       I-gpe       0.00      0.00      0.00        41
       I-nat       0.00      0.00      0.00        11
       I-org       0.61      0.17      0.27      4182
       I-per       0.52      0.10      0.17      4334
       I-tim       0.38      0.02      0.04      1593
           O       0.90      0.99      0.94    221962

    accuracy              

Results are not good. So we better attempt some other approaches. Thanks to internet, I found some useful content that I used for my understanding in this project.

I have used a post :http://www.depends-on-the-definition.com/introduction-named-entity-recognition-python/

-  where author solved problem by memorizing entities for words and then used a simple classification model to improve the results.

# 5)- Prediction by Memorization Memorization

Here sentence will be our X and tag will be y


In [0]:
from sklearn.base import BaseEstimator, TransformerMixin


class MemoryTagger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        '''
        Expects a list of words as X and a list of tags as y.
        '''
        voc = {}
        self.tags = []
        for x, t in zip(X, y):
            if t not in self.tags:
                self.tags.append(t)
            if x in voc:
                if t in voc[x]:
                    voc[x][t] += 1
                else:
                    voc[x][t] = 1
            else:
                voc[x] = {t: 1}
        self.memory = {}
        for k, d in voc.items():
            self.memory[k] = max(d, key=d.get)
    
    def predict(self, X, y=None):
        '''
        Predict the the tag from memory. If word is unknown, predict 'O'.
        '''
        return [self.memory.get(x, 'O') for x in X]

In [0]:
# initialize 
tagger = MemoryTagger()

In [0]:
tagger.fit(sent, tag)

# X being sent and y being tag

In [43]:
print(tagger.predict(sent))

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [44]:
tagger.tags

['O', 'B-geo', 'B-gpe']

# 6)- Cross Validation Method

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

In [0]:
words = data["Word"].values.tolist()
tags = data["Tag"].values.tolist()

In [0]:
pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)

In [48]:
accuracy = accuracy_score(y_pred=pred, y_true=tags)
print(accuracy)

0.9458875140071049


What if we need more detail then classification report would be more useful

In [49]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

              precision    recall  f1-score   support

       B-art       0.20      0.05      0.09       402
       B-eve       0.54      0.25      0.34       308
       B-geo       0.78      0.85      0.81     37644
       B-gpe       0.94      0.93      0.94     15870
       B-nat       0.42      0.28      0.33       201
       B-org       0.67      0.49      0.56     20143
       B-per       0.78      0.65      0.71     16990
       B-tim       0.87      0.77      0.82     20333
       I-art       0.04      0.01      0.01       297
       I-eve       0.39      0.12      0.18       253
       I-geo       0.73      0.58      0.65      7414
       I-gpe       0.62      0.45      0.52       198
       I-nat       0.00      0.00      0.00        51
       I-org       0.69      0.53      0.60     16784
       I-per       0.73      0.65      0.69     17251
       I-tim       0.58      0.13      0.21      6528
           O       0.97      0.99      0.98    887908

    accuracy              

# 7)- Back to ML model

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
def feature_map(word):
    '''Simple feature map.'''
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha()])

In [0]:
words = [feature_map(w) for w in data["Word"].values.tolist()]

In [53]:
words[:5]

[array([1, 0, 0, 9, 0, 1]),
 array([0, 1, 0, 2, 0, 1]),
 array([ 0,  1,  0, 13,  0,  1]),
 array([0, 1, 0, 4, 0, 1]),
 array([0, 1, 0, 7, 0, 1])]

In [0]:
pred = cross_val_predict(RandomForestClassifier(n_estimators=20),
                         X=words, y=tags, cv=5)

In [55]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       402
       B-eve       0.00      0.00      0.00       308
       B-geo       0.26      0.80      0.40     37644
       B-gpe       0.26      0.04      0.07     15870
       B-nat       0.00      0.00      0.00       201
       B-org       0.65      0.17      0.27     20143
       B-per       0.97      0.20      0.33     16990
       B-tim       0.29      0.32      0.30     20333
       I-art       0.00      0.00      0.00       297
       I-eve       0.00      0.00      0.00       253
       I-geo       0.00      0.00      0.00      7414
       I-gpe       0.00      0.00      0.00       198
       I-nat       0.00      0.00      0.00        51
       I-org       0.36      0.03      0.06     16784
       I-per       0.46      0.02      0.04     17251
       I-tim       0.50      0.06      0.11      6528
           O       0.97      0.98      0.97    887908

    accuracy              

# 7)- Feature Transformation

To improve results

In [0]:
from sklearn.preprocessing import LabelEncoder

class FeatureTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.memory_tagger = MemoryTagger()
        self.tag_encoder = LabelEncoder()
        self.pos_encoder = LabelEncoder()
        
    def fit(self, X, y):
        words = X["Word"].values.tolist()
        self.pos = X["POS"].values.tolist()
        tags = X["Tag"].values.tolist()
        self.memory_tagger.fit(words, tags)
        self.tag_encoder.fit(tags)
        self.pos_encoder.fit(self.pos)
        return self
    
    def transform(self, X, y=None):
        def pos_default(p):
            if p in self.pos:
                return self.pos_encoder.transform([p])[0]
            else:
                return -1
        
        pos = X["POS"].values.tolist()
        words = X["Word"].values.tolist()
        out = []
        for i in range(len(words)):
            w = words[i]
            p = pos[i]
            if i < len(words) - 1:
                wp = self.tag_encoder.transform(self.memory_tagger.predict([words[i+1]]))[0]
                posp = pos_default(pos[i+1])
            else:
                wp = self.tag_encoder.transform(['O'])[0]
                posp = pos_default(".")
            if i > 0:
                if words[i-1] != ".":
                    wm = self.tag_encoder.transform(self.memory_tagger.predict([words[i-1]]))[0]
                    posm = pos_default(pos[i-1])
                else:
                    wm = self.tag_encoder.transform(['O'])[0]
                    posm = pos_default(".")
            else:
                posm = pos_default(".")
                wm = self.tag_encoder.transform(['O'])[0]
            out.append(np.array([w.istitle(), w.islower(), w.isupper(), len(w), w.isdigit(), w.isalpha(),
                                 self.tag_encoder.transform(self.memory_tagger.predict([w]))[0],
                                 pos_default(p), wp, wm, posp, posm]))
        return out

### using pipeline

in order to utilize featuretransform + random forest at same. Else we can do one by one.

In [0]:
from sklearn.pipeline import Pipeline
pred = cross_val_predict(Pipeline([("feature_map", FeatureTransformer()), 
                                   ("clf", RandomForestClassifier(n_estimators=20, n_jobs=3))]),
                         X=data, y=tags, cv=5)

In [0]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)